<a href="https://colab.research.google.com/github/erik-777/NLP_Taller1/blob/main/7_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis de Sentimientos en reseñas de películas

# Version: Ivan Moran - Erik Vergara


Ahora pongamos en práctica algunos de estos conceptos en un caso más real. Para esta práctica vamos a hacer un análisis de sentimientos sobre unas reseñas de películas. Este caso sería una simple clasificación binaria y podemos utilizar cualquier modelo para ese fin, lo adicional aquí es el pre-procesamiento de las entradas de texto.

### Referencias
* [Natural Language Processing in Action](https://www.manning.com/books/natural-language-processing-in-action)

In [1]:
# Se instalan solo librerias necesarias
!pip install nltk==3.9.1

In [2]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/Sesion1/moviereviews.tsv

Empecemos por cargar el dataset:

In [6]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('./moviereviews.tsv', sep='\t')
reviews.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


## Limpieza de Datos

Antes de proceder con el análisis, realizamos un proceso básico de limpieza para garantizar la calidad del dataset.

Las acciones incluyen:
- Eliminación de valores nulos
- Eliminación de reseñas vacías
- Normalización básica del texto (remoción de espacios innecesarios)

Este paso es fundamental para evitar errores en etapas posteriores del procesamiento y modelado.

In [7]:
reviews.dropna(inplace=True)
reviews.review = reviews.review.apply(lambda r: r.strip())
blanks = reviews[reviews.review == ''].index
reviews.drop(blanks, inplace=True)

In [8]:
reviews[reviews.review == ''].index

Index([], dtype='int64')

In [9]:
reviews.label.value_counts()

,count
label,
neg,969
pos,969


## Aplicación de VADER para Análisis de Sentimiento

El dataset se encuentra balanceado, con aproximadamente mil ejemplos por clase.  

Para esta primera aproximación utilizaremos **VADER (Valence Aware Dictionary and sEntiment Reasoner)**, un modelo basado en reglas y léxico que permite estimar automáticamente la polaridad de un texto.

VADER está especialmente diseñado para análisis de sentimiento en textos cortos y es parte de la librería NLTK.  

Este modelo calcula cuatro métricas:

- **neg** → proporción de contenido negativo  
- **neu** → proporción de contenido neutral  
- **pos** → proporción de contenido positivo  
- **compound** → puntaje normalizado global entre -1 y 1  

El valor `compound` suele utilizarse como indicador final de polaridad.

In [10]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews['scores'] = reviews.review.apply(lambda r: sid.polarity_scores(r))
reviews.head()

,label,review,scores
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co..."
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com..."
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com..."
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co..."
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co..."


## Conversión de Puntajes a Etiquetas de Predicción

Una vez obtenido el puntaje `compound` de VADER, debemos transformarlo en una etiqueta binaria (`pos` / `neg`).

Para ello definimos un **threshold (umbral de decisión)** que determine a partir de qué valor consideramos que una reseña es positiva.

Por defecto, VADER recomienda:

- compound ≥ 0.05 → positivo
- compound ≤ -0.05 → negativo
- valores intermedios → neutral

En este caso, trabajaremos con una clasificación binaria, por lo que utilizaremos el umbral de 0.05 para separar clases positivas y negativas.

In [12]:
# Se ajusta Threshold a 0.05 para mejorar resultado en negativos
reviews['compound'] = reviews.scores.apply(lambda s: s['compound'])
# reviews['prediction'] = reviews['compound'].apply(lambda c: 'pos' if c > 0 else 'neg')
reviews['prediction'] = reviews['compound'].apply(lambda c: 'pos' if c > 0.05 else 'neg')
reviews.head()

,label,review,scores,compound,prediction
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125,neg
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618,neg
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951,pos
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972,pos
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484,neg


## Evaluación del Desempeño del Modelo

Una vez generadas las predicciones, evaluamos el desempeño del modelo comparando las etiquetas reales (`label`) contra las etiquetas predichas (`prediction`).

Utilizaremos métricas estándar de clasificación:

- Accuracy
- Precision
- Recall
- F1-score
- Matriz de confusión

In [13]:


from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Etiquetas reales y predichas
y_true = reviews.label.values
y_pred = reviews.prediction.values

# Métricas
acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)

print(f"Accuracy:\n{acc:.4f}\n")
print("Classification Report:\n", cr)
print("Confusion Matrix:\n", cm)

Accuracy:
0.6357

Classification Report:
               precision    recall  f1-score   support

         neg       0.72      0.44      0.55       969
         pos       0.60      0.83      0.70       969

    accuracy                           0.64      1938
   macro avg       0.66      0.64      0.62      1938
weighted avg       0.66      0.64      0.62      1938

Confusion Matrix:
 [[427 542]
 [164 805]]


## Conclusión del Modelo Basado en VADER

El modelo alcanza una accuracy de aproximadamente 63.6%, lo cual representa una mejora significativa frente a una línea base aleatoria (50%). Sin embargo, el desempeño aún es limitado.

El principal problema se observa en la clase negativa:

- Recall en negativos: 0.44  
- Recall en positivos: 0.83  

Esto indica que el modelo tiene dificultades para identificar correctamente reseñas negativas y tiende a clasificarlas como positivas.

En otras palabras, existe un sesgo hacia la clase positiva, lo cual se evidencia también en la matriz de confusión.

## Modelo Supervisado: TF-IDF + Logistic Regression

En esta sección implementamos un modelo supervisado de clasificación de texto.

A diferencia de VADER (modelo basado en reglas), ahora entrenaremos un modelo que aprende patrones directamente del dataset.

El pipeline incluye:

1. División train/test estratificada
2. Vectorización TF-IDF
3. Entrenamiento con Logistic Regression
4. Evaluación sobre el conjunto de prueba

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Separación train/test (estratificada)
X_train, X_test, y_train, y_test = train_test_split(
    reviews.review,
    reviews.label,
    test_size=0.2,
    random_state=42,
    stratify=reviews.label
)

# Vectorizador TF-IDF
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000,
    ngram_range=(1, 2)
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Modelo Logistic Regression
model = LogisticRegression(
    max_iter=1000,
    class_weight=None  # dataset balanceado
)

model.fit(X_train_tfidf, y_train)

# Predicciones sobre test
y_pred = model.predict(X_test_tfidf)

# Guardar predicciones en dataframe (solo para test)
reviews.loc[X_test.index, 'prediction_ml'] = y_pred

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Corrected approach: Use only the test set for evaluation of the TF-IDF model.
# `y_test` (from train_test_split in cell A8l_Wa1HCbjr) contains the true labels for the test set.
# `model.predict(X_test_tfidf)` re-generates the predictions from the TF-IDF Logistic Regression model
# specifically for the test set, ensuring consistency.

y_true_tfidf = y_test # True labels for the TF-IDF test set
y_pred_tfidf = model.predict(X_test_tfidf) # Predictions from the TF-IDF Logistic Regression model

acc = accuracy_score(y_true_tfidf, y_pred_tfidf)
cm = confusion_matrix(y_true_tfidf, y_pred_tfidf)
cr = classification_report(y_true_tfidf, y_pred_tfidf)

print(f"Accuracy (TF-IDF Model):\n{acc}\n")
print(f"Classification Report (TF-IDF Model):\n{cr}")
print(f"Confusion Matrix (TF-IDF Model):\n{cm}")

Accuracy (TF-IDF Model):
0.8273195876288659

Classification Report (TF-IDF Model):
              precision    recall  f1-score   support

         neg       0.83      0.82      0.83       194
         pos       0.83      0.83      0.83       194

    accuracy                           0.83       388
   macro avg       0.83      0.83      0.83       388
weighted avg       0.83      0.83      0.83       388

Confusion Matrix (TF-IDF Model):
[[160  34]
 [ 33 161]]


## Análisis de Resultados

El modelo supervisado alcanza una accuracy de aproximadamente **82.7%**, lo que representa una mejora significativa frente al modelo basado en reglas (VADER ≈ 63.6%).

### Métricas por clase

- Clase negativa:
  - Precision: 0.83
  - Recall: 0.82
  - F1-score: 0.83

- Clase positiva:
  - Precision: 0.83
  - Recall: 0.83
  - F1-score: 0.83

Observamos un desempeño balanceado entre ambas clases, sin sesgo evidente.

Porque el modelo supervisado:
- Aprende patrones específicos del dataset
- Usa bigramas (ngram_range=(1,2))
- No depende de un diccionario fijo
- Aprende correlaciones reales entre palabras y etiquetas

VADER:
- No aprende
- No se adapta al dominio
- Está optimizado para redes sociales, no reseñas largas

**Palabras influyentes**

In [16]:
feature_names = tfidf.get_feature_names_out()
coefs = model.coef_[0]

top_pos = sorted(zip(coefs, feature_names), reverse=True)[:15]
top_neg = sorted(zip(coefs, feature_names))[:15]

print("Top palabras positivas:")
for coef, word in top_pos:
    print(word)

print("\nTop palabras negativas:")
for coef, word in top_neg:
    print(word)

Top palabras positivas:
life
great
seen
truman
family
mulan
fun
jackie
quite
performance
excellent
true
gives
hilarious
perfect

Top palabras negativas:
bad
worst
plot
stupid
boring
supposed
movie
waste
reason
unfortunately
script
poor
godzilla
ridiculous
awful


**Modelo Sentence-Bert**

In [17]:
# ================================
# SENTENCE-BERT + Logistic Regression
# ================================

from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# 1️⃣ Separación train/test
X_train, X_test, y_train, y_test = train_test_split(
    reviews.review,
    reviews.label,
    test_size=0.2,
    random_state=42,
    stratify=reviews.label
)

# 2️⃣ Cargar modelo Sentence-BERT
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

# 3️⃣ Generar embeddings
X_train_embeddings = model_sbert.encode(
    X_train.tolist(),
    convert_to_numpy=True,
    show_progress_bar=True
)

X_test_embeddings = model_sbert.encode(
    X_test.tolist(),
    convert_to_numpy=True,
    show_progress_bar=True
)

# 4️⃣ Clasificador (Logistic Regression)
classifier = LogisticRegression(max_iter=1000)

classifier.fit(X_train_embeddings, y_train)

# 5️⃣ Predicciones
y_pred = classifier.predict(X_test_embeddings)

# Guardar predicciones en el dataframe (solo test)
reviews.loc[X_test.index, 'prediction_ml'] = y_pred



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/49 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [18]:
# ================================
# MÉTRICAS
# ================================

y_true = y_test.values

acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)

print(f"Accuracy:\n{acc}\n")
print(f"Classification Report:\n{cr}")
print(f"Confusion Matrix:\n{cm}")

Accuracy:
0.6984536082474226

Classification Report:
              precision    recall  f1-score   support

         neg       0.69      0.73      0.71       194
         pos       0.71      0.66      0.69       194

    accuracy                           0.70       388
   macro avg       0.70      0.70      0.70       388
weighted avg       0.70      0.70      0.70       388

Confusion Matrix:
[[142  52]
 [ 65 129]]


**Ejemplos:**

(Positivo)

The Housemaid is the real deal. It’s a slick, high-voltage thriller that takes a classic setup and turns it into a total masterclass in suspense. The lead is absolutely magnetic, and the twists are so sharp they actually land instead of just being "filler." It’s stylish, mean, and keeps you hooked from start to finish. Honestly? It’s a total knockout that makes the rest of the genre look lazy.

(Negativo)

The Housemaid is a tired exercise in predictability that mistakes recycled clichés for genuine suspense. While visually polished, its hollow script and telegraphed twists waste a talented cast, delivering a thriller that is as shallow as it is derivative. It’s a chore to watch a film so convinced of its own cleverness while offering nothing we haven't seen a dozen times before.

In [ ]:
# 1️⃣ Ingresar nuevo comentario
nuevo_comentario = input("Ingrese un comentario: ")

print("\nComentario ingresado:")
print(nuevo_comentario)
print("\n--- RESULTADOS ---")

# ==========================================
# 🔹 MODELO 1: NLTK (VADER)
# ==========================================

scores = sid.polarity_scores(nuevo_comentario)

# Regla estándar VADER
if scores['compound'] >= 0:
    pred_nltk = "pos"
else:
    pred_nltk = "neg"

print(f"NLTK (VADER): {pred_nltk} | Score: {scores['compound']:.4f}")


# ==========================================
# 🔹 MODELO 2: TF-IDF + Logistic Regression
# ==========================================

nuevo_tfidf = tfidf.transform([nuevo_comentario])
pred_tfidf = model.predict(nuevo_tfidf)

print(f"TF-IDF + Logistic Regression: {pred_tfidf}")


# ==========================================
# 🔹 MODELO 3: Sentence-BERT + Logistic
# ==========================================

nuevo_embedding = model_sbert.encode(
    [nuevo_comentario],
    convert_to_numpy=True
)

pred_sbert = classifier.predict(nuevo_embedding)[0]

print(f"Sentence-BERT + Logistic Regression: {pred_sbert}")

print("\n===================================")

Ingrese un comentario: Gracias

Comentario ingresado:
Gracias

--- RESULTADOS ---
NLTK (VADER): pos | Score: 0.0000
TF-IDF + Logistic Regression: ['pos']
Sentence-BERT + Logistic Regression: pos

